In [16]:
import os
import sys
import shutil
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import set_config

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering

from Import_csv import import_csv
imported_csv_path = import_csv()

df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_100722/65876671.py:31: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [17]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[2283906006, 1770895001, 8383554010, 116068500..."
1,Name,object,14,0.00,779583,"[Rocky Yau dba First Speed Trad, The Jedediah ..."
2,City,object,30,0.00,32581,"[MIAMI, INDIANAPOLIS, HELENA, ONTARIO, SANTA F..."
3,State,object,14,0.00,51,"[WI, AZ, CA, PA, GA]"
4,Zip,int64,0,0.00,33611,"[92844, 97211, 19401, 54656, 39051]"
5,Bank,object,1559,0.17,5802,"[GRANITE ST. ECONOMIC DEVEL COR, CEDAR RAPIDS ..."
6,BankState,object,1566,0.17,56,"[SD, GA, MO, NC, CA]"
7,NAICS,int64,0,0.00,1312,"[0, 722212, 524210]"
8,ApprovalDate,object,0,0.00,9859,"[6-Jun-08, 25-May-11, 25-Feb-04, 30-Jan-89, 9-..."
9,ApprovalFY,object,0,0.00,70,"[1992, 2001, 2007, 1993, 2004]"


In [18]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [19]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)
df.NAICS.value_counts()

NAICS
44-45    127251
81        72618
54        68170
31-33     68029
72        67600
23        66646
62        55366
42        48743
56        32685
48-49     22531
71        14640
53        13632
51        11379
52         9496
11         9005
61         6425
21         1851
22          663
55          257
92          229
Name: count, dtype: int64

In [20]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]
df.NewExist.value_counts()

NewExist
1.0    502830
2.0    193520
Name: count, dtype: int64

In [21]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [22]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

df['RevLineCr'].value_counts()

RevLineCr
N    484187
Y    195147
Name: count, dtype: int64

In [23]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

df['LowDoc'].value_counts()

LowDoc
N    608823
Y     66203
Name: count, dtype: int64

In [24]:
df['FranchiseCode'] = df['FranchiseCode'].apply(lambda x: 1 if x >= 1 else 0)

In [25]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [26]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [27]:
df.shape

(675026, 13)

In [28]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[71, 42, 44-45, 31-33, 62]"
1,Term,int64,0,0.0,377,"[84, 90, 240, 85, 120]"
2,NoEmp,int64,0,0.0,499,"[3, 2, 20, 12]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,216,"[0, 6]"
5,RetainedJob,int64,0,0.0,323,"[10, 12, 9, 0]"
6,FranchiseCode,int64,0,0.0,2,"[1, 0]"
7,UrbanRural,int64,0,0.0,3,"[2, 0, 1]"
8,RevLineCr,object,0,0.0,2,"[N, Y]"
9,LowDoc,object,0,0.0,2,[N]


In [29]:
df_to_csv(df, folder_path, 'SBA_Cleaned_0.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_0.csv


In [30]:
df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_100722/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [31]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [32]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [33]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [34]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [35]:
valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [36]:
valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [37]:
df['FranchiseCode'] = df['FranchiseCode'].apply(lambda x: 1 if x >= 1 else 0)

In [38]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [39]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [40]:
df.shape

(460816, 13)

In [41]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[23, 42, 44-45, 53]"
1,Term,int64,0,0.0,373,"[84, 96, 78]"
2,NoEmp,int64,0,0.0,432,"[6, 3, 1, 256, 12]"
3,NewExist,float64,0,0.0,2,"[2.0, 1.0]"
4,CreateJob,int64,0,0.0,192,[0]
5,RetainedJob,int64,0,0.0,276,"[0, 1, 4]"
6,FranchiseCode,int64,0,0.0,2,"[0, 1]"
7,UrbanRural,int64,0,0.0,3,"[0, 1]"
8,RevLineCr,object,0,0.0,2,"[Y, N]"
9,LowDoc,object,0,0.0,2,[N]


In [42]:
df_to_csv(df, folder_path, 'SBA_Cleaned_1.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_1.csv


In [43]:
df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_100722/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [44]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [45]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [46]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [47]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [48]:
samples_bad_lol = df[(df['ChgOffDate'].notna())]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [49]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & ((df['ChgOffPrinGr'] != 0.0) | (df['ChgOffPrinGr'] > 0.0))]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [50]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [51]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [52]:
df['FranchiseCode'] = df['FranchiseCode'].apply(lambda x: 1 if x >= 1 else 0)

In [53]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [54]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [55]:
df.shape

(675026, 13)

In [56]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[23, 81, 53]"
1,Term,int64,0,0.0,377,"[84, 72, 60]"
2,NoEmp,int64,0,0.0,499,"[25, 8, 4, 2]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,216,"[2, 0, 5, 11, 1]"
5,RetainedJob,int64,0,0.0,323,[0]
6,FranchiseCode,int64,0,0.0,2,"[1, 0]"
7,UrbanRural,int64,0,0.0,3,"[1, 0]"
8,RevLineCr,object,0,0.0,2,[N]
9,LowDoc,object,0,0.0,2,[N]


In [57]:
df_to_csv(df, folder_path, 'SBA_Cleaned_2.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_2.csv
